# Task 1 of Assignment 2

**Objective:** Train separate models for pathology groups to leverage shared or unique characteristics

**Split Ratio:** 5:3:2 (50% training, 30% validation, 20% testing), applied independently per group

**Pathology Grouping:**

- Lung Diseases: Atelectasis, Consolidation, Edema, Lung Opacity, Pneumonia, Lung Lesion (6)

- Pleural-Related: Effusion, Pleural Other, Pneumothorax (3)

- Cardiac and Mediastinal: Cardiomegaly, Enlarged Cardiomediastinum (2)

- Fracture: Fracture (1)

- Support Devices: Support Devices (1)

**Key Features:**

- Data split with fixed seed (seed=42) for consistency
- NaN values converted to 0, unrecognized pathologies defaulted to 0
- Outputs dictionary with 5 groups, each containing train/validation/test datasets

### 1. Load Data

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
from tqdm import tqdm
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set TensorFlow logging level to suppress unnecessary output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Helper function: Load embeddings from TFRecord files
def load_embedding(embedding_path):
    raw_dataset = tf.data.TFRecordDataset([embedding_path])
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        embedding_feature = example.features.feature['embedding']
        embedding_values = embedding_feature.float_list.value
    embedding = torch.tensor(embedding_values)
    if torch.isnan(embedding).any() or torch.isinf(embedding).any():
        raise ValueError(f"NaN or Inf found in embedding: {embedding_path}")
    return embedding

# Custom dataset class for MIMIC chest X-ray embeddings with grouped pathologies
class MIMIC_Embed_Dataset(Dataset):
    pathology_groups = {
        "Lung Diseases": ["Atelectasis", "Consolidation", "Edema", "Lung Opacity", "Pneumonia", "Lung Lesion"],
        "Pleural-Related": ["Pleural Effusion", "Pleural Other", "Pneumothorax"],
        "Cardiac and Mediastinal": ["Cardiomegaly", "Enlarged Cardiomediastinum"],
        "Fracture": ["Fracture"],
        "Support Devices": ["Support Devices"]
    }

    def __init__(self, embedpath, csvpath, metacsvpath, group_name, views=["PA"], mode="train", seed=42, split_ratio=[0.5, 0.3, 0.2], unique_patients=True):
        super().__init__()
        np.random.seed(seed)
        self.group_name = group_name
        self.pathologies = self.pathology_groups[group_name]
        self.mode = mode
        self.embedpath = embedpath
        self.csv = pd.read_csv(csvpath)
        self.metacsv = pd.read_csv(metacsvpath)
        self.split_ratio = split_ratio

        self.csv = self.csv.set_index(["subject_id", "study_id"])
        self.metacsv = self.metacsv.set_index(["subject_id", "study_id"])
        self.csv = self.csv.join(self.metacsv).reset_index()

        self.csv["view"] = self.csv["ViewPosition"]
        self.limit_to_selected_views(views)

        if unique_patients:
            self.csv = self.csv.groupby("subject_id").first().reset_index()

        n_row = self.csv.shape[0]
        if self.mode == "train":
            self.csv = self.csv[:int(n_row * self.split_ratio[0])]
        elif self.mode == "valid":
            self.csv = self.csv[int(n_row * self.split_ratio[0]):int(n_row * (self.split_ratio[0] + self.split_ratio[1]))]
        elif self.mode == "test":
            self.csv = self.csv[int(n_row * (self.split_ratio[0] + self.split_ratio[1])):]
        else:
            raise ValueError(f"mode must be one of [train, valid, test], got {self.mode}")

        healthy = self.csv["No Finding"] == 1
        labels = []
        for pathology in self.pathologies:
            if pathology in self.csv.columns:
                self.csv.loc[healthy, pathology] = 0
                mask = self.csv[pathology]
            else:
                mask = pd.Series(0, index=self.csv.index)
            labels.append(mask.values)
        self.labels = np.asarray(labels).T.astype(np.float32)
        self.labels[np.isnan(self.labels)] = 0
        self.labels = np.where((self.labels != 0) & (self.labels != 1), 0, self.labels)
        

        self.csv = self.csv.reset_index(drop=True)
        self.csv["offset_day_int"] = self.csv["StudyDate"]
        self.csv["patientid"] = self.csv["subject_id"].astype(str)

    def limit_to_selected_views(self, views):
        if not isinstance(views, list):
            views = [views]
        if '*' in views:
            views = ["*"]
        self.views = views
        self.csv["view"] = self.csv["view"].fillna("UNKNOWN")
        if "*" not in views:
            self.csv = self.csv[self.csv["view"].isin(self.views)]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {}
        sample["idx"] = idx
        sample["lab"] = self.labels[idx]
        subjectid = str(self.csv.iloc[idx]["subject_id"])
        studyid = str(self.csv.iloc[idx]["study_id"])
        dicom_id = str(self.csv.iloc[idx]["dicom_id"])
        embed_file = os.path.join(self.embedpath, f"p{subjectid[:2]}", f"p{subjectid}", f"s{studyid}", f"{dicom_id}.tfrecord")
        sample["embedding"] = load_embedding(embed_file)
        return sample

# Function to initialize datasets
def initialize_mimic_datasets(embedpath, csvpath, metacsvpath, seed=42, split_ratio=[0.5, 0.3, 0.2]):
    dataset_dict = {}
    group_names = ["Lung Diseases", "Pleural-Related", "Cardiac and Mediastinal", "Fracture", "Support Devices"]
    
    for group in group_names:
        dataset_dict[group] = {
            "train": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="train", seed=seed, split_ratio=split_ratio),
            "valid": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="valid", seed=seed, split_ratio=split_ratio),
            "test": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="test", seed=seed, split_ratio=split_ratio)
        }
    
    return dataset_dict

# Function to print dataset details
def print_dataset_details(datasets, num_samples=3):
    """
    Print details about the datasets to understand their structure and content.
    
    Args:
        datasets (dict): Dictionary of datasets for each pathology group
        num_samples (int): Number of samples to print per dataset
    """
    for group_name, splits in datasets.items():
        print(f"\n=== {group_name} ===")
        for split_name, dataset in splits.items():
            print(f"\n{split_name} Dataset:")
            print(f"Size: {len(dataset)} samples")
            print(f"Pathologies: {dataset.pathologies}")
            print(f"Label shape: {dataset.labels.shape}")
            
            # Print label distribution
            label_means = np.mean(dataset.labels, axis=0)
            print("Label distribution (mean per pathology):")
            for pathology, mean in zip(dataset.pathologies, label_means):
                print(f"  {pathology}: {mean:.3f}")
            
            # Print a few samples
            print(f"\nFirst {min(num_samples, len(dataset))} samples:")
            for i in range(min(num_samples, len(dataset))):
                sample = dataset[i]
                print(f"Sample {i}:")
                print(f"  Index: {sample['idx']}")
                print(f"  Labels: {sample['lab']}")
                print(f"  Embedding shape: {sample['embedding'].shape}")
                print(f"  Embedding (first 5 values): {sample['embedding'][:5]}")
            print("-" * 50)

# Example usage
embedpath = "/Users/prestonlim/Documents/phd/sph6004/assignment2/data/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files"
csvpath = "/Users/prestonlim/Documents/phd/sph6004/assignment2/data/mimic-cxr-2.0.0-chexpert.csv"
metacsvpath = "/Users/prestonlim/Documents/phd/sph6004/assignment2/data/mimic-cxr-2.0.0-metadata.csv"

datasets = initialize_mimic_datasets(
    embedpath=embedpath,
    csvpath=csvpath,
    metacsvpath=metacsvpath,
    seed=42,
    split_ratio=[0.5, 0.3, 0.2]
)

# Print details about the datasets
print_dataset_details(datasets, num_samples=3)




=== Lung Diseases ===

train Dataset:
Size: 22814 samples
Pathologies: ['Atelectasis', 'Consolidation', 'Edema', 'Lung Opacity', 'Pneumonia', 'Lung Lesion']
Label shape: (22814, 6)
Label distribution (mean per pathology):
  Atelectasis: 0.088
  Consolidation: 0.018
  Edema: 0.025
  Lung Opacity: 0.118
  Pneumonia: 0.046
  Lung Lesion: 0.029

First 3 samples:
Sample 0:
  Index: 0
  Labels: [0. 0. 0. 0. 0. 0.]
  Embedding shape: torch.Size([1376])
  Embedding (first 5 values): tensor([ 0.1257, -1.8030,  1.2843, -1.8088,  0.1278])
Sample 1:
  Index: 1
  Labels: [0. 0. 0. 0. 0. 0.]
  Embedding shape: torch.Size([1376])
  Embedding (first 5 values): tensor([-0.0282, -1.4541,  1.1770, -2.3273,  0.1422])
Sample 2:
  Index: 2
  Labels: [0. 0. 0. 0. 0. 0.]
  Embedding shape: torch.Size([1376])
  Embedding (first 5 values): tensor([-0.7325, -2.4499,  1.2821, -2.6194,  0.5554])
--------------------------------------------------

valid Dataset:
Size: 13688 samples
Pathologies: ['Atelectasis', 'Co

Below is a use case for calling data, the content on the right side of the equal sign can be directly called.

In [2]:
# Load Lung Diseases
lung_diseases_train = datasets["Lung Diseases"]["train"]
lung_diseases_valid = datasets["Lung Diseases"]["valid"]
lung_diseases_test = datasets["Lung Diseases"]["test"]
# Load Pleural-Related
pleural_related_train = datasets["Pleural-Related"]["train"]
pleural_related_valid = datasets["Pleural-Related"]["valid"]
pleural_related_test = datasets["Pleural-Related"]["test"]
# Load Cardiac and Mediastinal
cardiac_mediastinal_train = datasets["Cardiac and Mediastinal"]["train"]
cardiac_mediastinal_valid = datasets["Cardiac and Mediastinal"]["valid"]
cardiac_mediastinal_test = datasets["Cardiac and Mediastinal"]["test"]  
#Load Fracture
fracture_train = datasets["Fracture"]["train"]
fracture_valid = datasets["Fracture"]["valid"]
fracture_test = datasets["Fracture"]["test"]
#Load Support Devices
support_devices_train = datasets["Support Devices"]["train"]
support_devices_valid = datasets["Support Devices"]["valid"]
support_devices_test = datasets["Support Devices"]["test"]

In [3]:
lung_diseases_X_train = np.stack([sample['embedding'].numpy() for sample in lung_diseases_train])  # shape: [n_samples, embedding_dim]
lung_diseases_y_train = np.stack([sample['lab'] for sample in lung_diseases_train])               # shape: [n_samples, n_labels]

pleural_related_X_train = np.stack([sample['embedding'].numpy() for sample in pleural_related_train])  # shape: [n_samples, embedding_dim]
pleural_related_y_train = np.stack([sample['lab'] for sample in pleural_related_train])               # shape: [n_samples, n_labels]

pleural_related_X_train = np.stack([sample['embedding'].numpy() for sample in pleural_related_train])  # shape: [n_samples, embedding_dim]
pleural_related_y_train = np.stack([sample['lab'] for sample in pleural_related_train])               # shape: [n_samples, n_labels]

cardiac_mediastinal_X_train = np.stack([sample['embedding'].numpy() for sample in cardiac_mediastinal_train])  # shape: [n_samples, embedding_dim]
cardiac_mediastinal_y_train = np.stack([sample['lab'] for sample in cardiac_mediastinal_train])

fracture_X_train = np.stack([sample['embedding'].numpy() for sample in fracture_train])  # shape: [n_samples, embedding_dim]
fracture_y_train = np.stack([sample['lab'] for sample in fracture_train])     

support_devices_X_train = np.stack([sample['embedding'].numpy() for sample in support_devices_train])  # shape: [n_samples, embedding_dim]
support_devices_y_train = np.stack([sample['lab'] for sample in support_devices_train])     

In [4]:
lung_diseases_X_valid = np.stack([sample['embedding'].numpy() for sample in lung_diseases_valid])  # shape: [n_samples, embedding_dim]
lung_diseases_y_valid = np.stack([sample['lab'] for sample in lung_diseases_valid])               # shape: [n_samples, n_labels]

pleural_related_X_valid = np.stack([sample['embedding'].numpy() for sample in pleural_related_valid])  # shape: [n_samples, embedding_dim]
pleural_related_y_valid = np.stack([sample['lab'] for sample in pleural_related_valid])               # shape: [n_samples, n_labels]

pleural_related_X_valid = np.stack([sample['embedding'].numpy() for sample in pleural_related_valid])  # shape: [n_samples, embedding_dim]
pleural_related_y_valid = np.stack([sample['lab'] for sample in pleural_related_valid])               # shape: [n_samples, n_labels]

cardiac_mediastinal_X_valid = np.stack([sample['embedding'].numpy() for sample in cardiac_mediastinal_valid])  # shape: [n_samples, embedding_dim]
cardiac_mediastinal_y_valid = np.stack([sample['lab'] for sample in cardiac_mediastinal_valid])

fracture_X_valid = np.stack([sample['embedding'].numpy() for sample in fracture_valid])  # shape: [n_samples, embedding_dim]
fracture_y_valid = np.stack([sample['lab'] for sample in fracture_valid])     

support_devices_X_valid = np.stack([sample['embedding'].numpy() for sample in support_devices_valid])  # shape: [n_samples, embedding_dim]
support_devices_y_valid = np.stack([sample['lab'] for sample in support_devices_valid])     

In [3]:
lung_diseases_X_test = np.stack([sample['embedding'].numpy() for sample in lung_diseases_test])  # shape: [n_samples, embedding_dim]
lung_diseases_y_test = np.stack([sample['lab'] for sample in lung_diseases_test])               # shape: [n_samples, n_labels]

pleural_related_X_test = np.stack([sample['embedding'].numpy() for sample in pleural_related_test])  # shape: [n_samples, embedding_dim]
pleural_related_y_test = np.stack([sample['lab'] for sample in pleural_related_test])               # shape: [n_samples, n_labels]

pleural_related_X_test = np.stack([sample['embedding'].numpy() for sample in pleural_related_test])  # shape: [n_samples, embedding_dim]
pleural_related_y_test = np.stack([sample['lab'] for sample in pleural_related_test])               # shape: [n_samples, n_labels]

cardiac_mediastinal_X_test = np.stack([sample['embedding'].numpy() for sample in cardiac_mediastinal_test])  # shape: [n_samples, embedding_dim]
cardiac_mediastinal_y_test = np.stack([sample['lab'] for sample in cardiac_mediastinal_test])

fracture_X_test = np.stack([sample['embedding'].numpy() for sample in fracture_test])  # shape: [n_samples, embedding_dim]
fracture_y_test = np.stack([sample['lab'] for sample in fracture_test])     

support_devices_X_test = np.stack([sample['embedding'].numpy() for sample in support_devices_test])  # shape: [n_samples, embedding_dim]
support_devices_y_test = np.stack([sample['lab'] for sample in support_devices_test])     

In [5]:
pleural_related_train[100]

{'idx': 100,
 'lab': array([0., 0., 0.], dtype=float32),
 'embedding': tensor([-0.3845, -1.0236,  1.4351,  ...,  0.0020,  1.4400, -0.4826])}

In [4]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, multilabel_confusion_matrix, balanced_accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler

import joblib

In [5]:
lung_diseases_assignment_dir = "/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models_lung_diseases"
lung_diseases_vanilla_logreg_model_path = f"{lung_diseases_assignment_dir}/vanilla_logreg_model.pkl"
lung_diseases_l1_logreg_model_path = f"{lung_diseases_assignment_dir}/l1_logreg_model.pkl"
lung_diseases_l2_logreg_model_path = f"{lung_diseases_assignment_dir}/l2_logreg_model.pkl"
lung_diseases_elasticnet_logreg_model_path = f"{lung_diseases_assignment_dir}/elasticnet_logreg_model.pkl"

pleural_related_assignment_dir = "/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models_pleural_related"
pleural_related_vanilla_logreg_model_path = f"{pleural_related_assignment_dir}/vanilla_logreg_model.pkl"
pleural_related_l1_logreg_model_path = f"{pleural_related_assignment_dir}/l1_logreg_model.pkl"
pleural_related_l2_logreg_model_path = f"{pleural_related_assignment_dir}/l2_logreg_model.pkl"
pleural_related_elasticnet_logreg_model_path = f"{pleural_related_assignment_dir}/elasticnet_logreg_model.pkl"

cardiac_mediastinal_assignment_dir = "/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models_cardiac_mediastinal"
cardiac_mediastinal_vanilla_logreg_model_path = f"{cardiac_mediastinal_assignment_dir}/vanilla_logreg_model.pkl"
cardiac_mediastinal_l1_logreg_model_path = f"{cardiac_mediastinal_assignment_dir}/l1_logreg_model.pkl"
cardiac_mediastinal_l2_logreg_model_path = f"{cardiac_mediastinal_assignment_dir}/l2_logreg_model.pkl"
cardiac_mediastinal_elasticnet_logreg_model_path = f"{cardiac_mediastinal_assignment_dir}/elasticnet_logreg_model.pkl"

fracture_assignment_dir = "/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models_fracture"
fracture_vanilla_logreg_model_path = f"{fracture_assignment_dir}/vanilla_logreg_model.pkl"
fracture_l1_logreg_model_path = f"{fracture_assignment_dir}/l1_logreg_model.pkl"
fracture_l2_logreg_model_path = f"{fracture_assignment_dir}/l2_logreg_model.pkl"
fracture_elasticnet_logreg_model_path = f"{fracture_assignment_dir}/elasticnet_logreg_model.pkl"

support_devices_assignment_dir = "/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models_support_devices"
support_devices_vanilla_logreg_model_path = f"{support_devices_assignment_dir}/vanilla_logreg_model.pkl"
support_devices_l1_logreg_model_path = f"{support_devices_assignment_dir}/l1_logreg_model.pkl"
support_devices_l2_logreg_model_path = f"{support_devices_assignment_dir}/l2_logreg_model.pkl"
support_devices_elasticnet_logreg_model_path = f"{support_devices_assignment_dir}/elasticnet_logreg_model.pkl"

In [9]:
lung_diseases_vanilla_logreg_model = MultiOutputClassifier(
    LogisticRegression(
        penalty=None,
        solver='lbfgs',
        max_iter=10000,
        n_jobs=-1,
        random_state=10
    )
).fit(lung_diseases_X_train, lung_diseases_y_train)

joblib.dump(lung_diseases_vanilla_logreg_model, lung_diseases_vanilla_logreg_model_path)

y_pred_logreg = lung_diseases_vanilla_logreg_model.predict(lung_diseases_X_valid)
print("Lung disease logistic Regression:")
print(classification_report(lung_diseases_y_valid, y_pred_logreg, zero_division=0, target_names=lung_diseases_valid.pathologies))

Lung disease logistic Regression:
               precision    recall  f1-score   support

  Atelectasis       0.41      0.24      0.30      1211
Consolidation       0.06      0.14      0.08       243
        Edema       0.16      0.27      0.20       356
 Lung Opacity       0.42      0.23      0.30      1612
    Pneumonia       0.25      0.10      0.14       685
  Lung Lesion       0.18      0.11      0.14       392

    micro avg       0.28      0.20      0.23      4499
    macro avg       0.25      0.18      0.19      4499
 weighted avg       0.33      0.20      0.24      4499
  samples avg       0.05      0.04      0.04      4499



In [10]:
pleural_related_vanilla_logreg_model = MultiOutputClassifier(
    LogisticRegression(
        penalty=None,
        solver='lbfgs',
        max_iter=10000,
        n_jobs=-1,
        random_state=10
    )
).fit(pleural_related_X_train, pleural_related_y_train)

joblib.dump(pleural_related_vanilla_logreg_model, pleural_related_vanilla_logreg_model_path)

y_pred_logreg = pleural_related_vanilla_logreg_model.predict(pleural_related_X_valid)
print("Lung disease logistic Regression:")
print(classification_report(pleural_related_y_valid, y_pred_logreg, zero_division=0, target_names=pleural_related_valid.pathologies))

Lung disease logistic Regression:
                  precision    recall  f1-score   support

Pleural Effusion       0.55      0.45      0.49      1121
   Pleural Other       0.07      0.14      0.09       123
    Pneumothorax       0.12      0.18      0.14       175

       micro avg       0.39      0.39      0.39      1419
       macro avg       0.25      0.25      0.24      1419
    weighted avg       0.46      0.39      0.42      1419
     samples avg       0.04      0.04      0.04      1419



In [11]:
cardiac_mediastinal_vanilla_logreg_model = MultiOutputClassifier(
    LogisticRegression(
        penalty=None,
        solver='lbfgs',
        max_iter=10000,
        n_jobs=-1,
        random_state=10
    )
).fit(cardiac_mediastinal_X_train, cardiac_mediastinal_y_train)

joblib.dump(cardiac_mediastinal_vanilla_logreg_model, cardiac_mediastinal_vanilla_logreg_model_path)

y_pred_logreg = cardiac_mediastinal_vanilla_logreg_model.predict(cardiac_mediastinal_X_valid)
print("Lung disease logistic Regression:")
print(classification_report(cardiac_mediastinal_y_valid, y_pred_logreg, zero_division=0, target_names=cardiac_mediastinal_valid.pathologies))

Lung disease logistic Regression:
                            precision    recall  f1-score   support

              Cardiomegaly       0.40      0.26      0.31      1085
Enlarged Cardiomediastinum       0.03      0.06      0.04       189

                 micro avg       0.26      0.23      0.24      1274
                 macro avg       0.21      0.16      0.17      1274
              weighted avg       0.34      0.23      0.27      1274
               samples avg       0.02      0.02      0.02      1274



In [45]:
fracture_vanilla_logreg_model = LogisticRegression(
    penalty=None,
    solver='lbfgs',
    max_iter=10000,
    n_jobs=-1,
    random_state=10
).fit(fracture_X_train, fracture_y_train.ravel())

joblib.dump(fracture_vanilla_logreg_model, fracture_vanilla_logreg_model_path)

y_pred_logreg = fracture_vanilla_logreg_model.predict(fracture_X_valid)
print("Fracture logistic Regression:")
print(classification_report(
    fracture_y_valid.ravel(),
    y_pred_logreg,
    labels=[1],  # Only show metrics for class "1" (Fracture)
    target_names=["Fracture"],  # Optional: single class name
    zero_division=0
))


Fracture logistic Regression:
              precision    recall  f1-score   support

    Fracture       0.07      0.06      0.06       292

   micro avg       0.07      0.06      0.06       292
   macro avg       0.07      0.06      0.06       292
weighted avg       0.07      0.06      0.06       292



In [46]:
support_devices_vanilla_logreg_model = LogisticRegression(
    penalty=None,
    solver='lbfgs',
    max_iter=10000,
    n_jobs=-1,
    random_state=10
).fit(support_devices_X_train, support_devices_y_train.ravel())

joblib.dump(support_devices_vanilla_logreg_model, support_devices_vanilla_logreg_model_path)

y_pred_logreg = support_devices_vanilla_logreg_model.predict(support_devices_X_valid)
print("Support devices logistic Regression:")
print(classification_report(
  support_devices_y_valid.ravel(),
  y_pred_logreg,
  labels=[1],
  target_names=["Support devices"],
  zero_division=0))

Support devices logistic Regression:
                 precision    recall  f1-score   support

Support devices       0.28      0.23      0.26       572

      micro avg       0.28      0.23      0.26       572
      macro avg       0.28      0.23      0.26       572
   weighted avg       0.28      0.23      0.26       572



In [29]:
lung_diseases_l1_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
      penalty='l1',
      solver='liblinear',
      Cs=10,
      max_iter=10000,\
      n_jobs=-1,
      cv=5
  )
).fit(lung_diseases_X_train, lung_diseases_y_train)

joblib.dump(lung_diseases_l1_logreg_model, lung_diseases_l1_logreg_model_path)

y_pred_logreg = lung_diseases_l1_logreg_model.predict(lung_diseases_X_valid)
print("Lung disease logistic Regression:")
print(classification_report(lung_diseases_y_valid, y_pred_logreg, zero_division=0, target_names=lung_diseases_valid.pathologies))

Lung disease logistic Regression:
               precision    recall  f1-score   support

  Atelectasis       0.53      0.16      0.24      1211
Consolidation       0.00      0.00      0.00       243
        Edema       0.66      0.11      0.19       356
 Lung Opacity       0.52      0.14      0.21      1612
    Pneumonia       0.69      0.03      0.06       685
  Lung Lesion       0.00      0.00      0.00       392

    micro avg       0.54      0.10      0.17      4499
    macro avg       0.40      0.07      0.12      4499
 weighted avg       0.49      0.10      0.17      4499
  samples avg       0.03      0.02      0.02      4499



In [30]:
pleural_related_l1_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
      penalty='l1',
      solver='liblinear',
      Cs=10,
      max_iter=10000,
      n_jobs=-1,
      cv=5
  )
).fit(pleural_related_X_train, pleural_related_y_train)

joblib.dump(pleural_related_l1_logreg_model, pleural_related_l1_logreg_model_path)

y_pred_logreg = pleural_related_l1_logreg_model.predict(pleural_related_X_valid)
print("Pleural related L1 logistic Regression:")
print(classification_report(pleural_related_y_valid, y_pred_logreg, zero_division=0, target_names=pleural_related_valid.pathologies))

Pleural related L1 logistic Regression:
                  precision    recall  f1-score   support

Pleural Effusion       0.68      0.45      0.54      1121
   Pleural Other       0.00      0.00      0.00       123
    Pneumothorax       0.00      0.00      0.00       175

       micro avg       0.68      0.35      0.47      1419
       macro avg       0.23      0.15      0.18      1419
    weighted avg       0.54      0.35      0.43      1419
     samples avg       0.04      0.03      0.04      1419



In [31]:
cardiac_mediastinal_l1_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
      penalty='l1',
      solver='liblinear',
      Cs=10,
      max_iter=10000,
      n_jobs=-1,
      cv=5
  )
).fit(cardiac_mediastinal_X_train, cardiac_mediastinal_y_train)

joblib.dump(cardiac_mediastinal_l1_logreg_model, cardiac_mediastinal_l1_logreg_model_path)

y_pred_logreg = cardiac_mediastinal_l1_logreg_model.predict(cardiac_mediastinal_X_valid)
print("Cardiac mediastinal L1 logistic Regression:")
print(classification_report(cardiac_mediastinal_y_valid, y_pred_logreg, zero_division=0, target_names=cardiac_mediastinal_valid.pathologies))

Cardiac mediastinal L1 logistic Regression:
                            precision    recall  f1-score   support

              Cardiomegaly       0.53      0.18      0.27      1085
Enlarged Cardiomediastinum       0.00      0.00      0.00       189

                 micro avg       0.53      0.15      0.24      1274
                 macro avg       0.26      0.09      0.13      1274
              weighted avg       0.45      0.15      0.23      1274
               samples avg       0.01      0.01      0.01      1274



In [47]:
fracture_l1_logreg_model = LogisticRegressionCV(
  penalty='l1',
  solver='liblinear',
  Cs=10,
  max_iter=10000,
  n_jobs=-1,
  cv=5
).fit(fracture_X_train, fracture_y_train.ravel())

joblib.dump(fracture_l1_logreg_model, fracture_l1_logreg_model_path)

y_pred_logreg = fracture_l1_logreg_model.predict(fracture_X_valid)
print("Fracture L1 logistic Regression:")
print(classification_report(
    fracture_y_valid.ravel(),
    y_pred_logreg,
    labels=[1],
    target_names=["Fracture"],
    zero_division=0
))

Fracture L1 logistic Regression:
              precision    recall  f1-score   support

    Fracture       0.00      0.00      0.00       292

   micro avg       0.00      0.00      0.00       292
   macro avg       0.00      0.00      0.00       292
weighted avg       0.00      0.00      0.00       292



In [48]:
support_devices_l1_logreg_model = LogisticRegressionCV(
      penalty='l1',
      solver='liblinear',
      Cs=10,
      max_iter=10000,
      n_jobs=-1,
      cv=5
).fit(support_devices_X_train, support_devices_y_train.ravel())

joblib.dump(support_devices_l1_logreg_model, support_devices_l1_logreg_model_path)

y_pred_logreg = support_devices_l1_logreg_model.predict(support_devices_X_valid)
print("Support devices L1 logistic Regression:")
print(classification_report(
  support_devices_y_valid.ravel(),
  y_pred_logreg,
  labels=[1],
  target_names=["Support devices"],
  zero_division=0))

Support devices L1 logistic Regression:
                 precision    recall  f1-score   support

Support devices       0.47      0.08      0.13       572

      micro avg       0.47      0.08      0.13       572
      macro avg       0.47      0.08      0.13       572
   weighted avg       0.47      0.08      0.13       572



In [34]:
lung_diseases_l2_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
    penalty='l2',
    solver='liblinear',
    Cs=10,
    max_iter=10000,
    n_jobs=-1,
    cv=5
  )
).fit(lung_diseases_X_train, lung_diseases_y_train)

joblib.dump(lung_diseases_l2_logreg_model, lung_diseases_l2_logreg_model_path)

y_pred_logreg = lung_diseases_l2_logreg_model.predict(lung_diseases_X_valid)
print("Lung disease logistic Regression:")
print(classification_report(lung_diseases_y_valid, y_pred_logreg, zero_division=0, target_names=lung_diseases_valid.pathologies))

Lung disease logistic Regression:
               precision    recall  f1-score   support

  Atelectasis       0.50      0.17      0.25      1211
Consolidation       0.00      0.00      0.00       243
        Edema       0.59      0.15      0.23       356
 Lung Opacity       0.52      0.12      0.20      1612
    Pneumonia       0.75      0.02      0.04       685
  Lung Lesion       0.00      0.00      0.00       392

    micro avg       0.52      0.10      0.17      4499
    macro avg       0.39      0.08      0.12      4499
 weighted avg       0.48      0.10      0.16      4499
  samples avg       0.03      0.02      0.02      4499



In [35]:
pleural_related_l2_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
    penalty='l2',
    solver='liblinear',
    Cs=10,
    max_iter=10000,
    n_jobs=-1,
    cv=5
  )
).fit(pleural_related_X_train, pleural_related_y_train)

joblib.dump(pleural_related_l2_logreg_model, pleural_related_l2_logreg_model_path)

y_pred_logreg = pleural_related_l2_logreg_model.predict(pleural_related_X_valid)
print("Pleural related L2 logistic Regression:")
print(classification_report(pleural_related_y_valid, y_pred_logreg, zero_division=0, target_names=pleural_related_valid.pathologies))

Pleural related L2 logistic Regression:
                  precision    recall  f1-score   support

Pleural Effusion       0.68      0.47      0.56      1121
   Pleural Other       0.00      0.00      0.00       123
    Pneumothorax       0.00      0.00      0.00       175

       micro avg       0.68      0.37      0.48      1419
       macro avg       0.23      0.16      0.19      1419
    weighted avg       0.54      0.37      0.44      1419
     samples avg       0.04      0.04      0.04      1419



In [36]:
cardiac_mediastinal_l2_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
    penalty='l2',
    solver='liblinear',
    Cs=10,
    max_iter=10000,
    n_jobs=-1,
    cv=5
  )
).fit(cardiac_mediastinal_X_train, cardiac_mediastinal_y_train)

joblib.dump(cardiac_mediastinal_l2_logreg_model, cardiac_mediastinal_l2_logreg_model_path)

y_pred_logreg = cardiac_mediastinal_l2_logreg_model.predict(cardiac_mediastinal_X_valid)
print("Cardiac mediastinal L2 logistic Regression:")
print(classification_report(cardiac_mediastinal_y_valid, y_pred_logreg, zero_division=0, target_names=cardiac_mediastinal_valid.pathologies))

Cardiac mediastinal L2 logistic Regression:
                            precision    recall  f1-score   support

              Cardiomegaly       0.66      0.07      0.13      1085
Enlarged Cardiomediastinum       0.00      0.00      0.00       189

                 micro avg       0.66      0.06      0.11      1274
                 macro avg       0.33      0.04      0.07      1274
              weighted avg       0.56      0.06      0.11      1274
               samples avg       0.01      0.01      0.01      1274



In [49]:
fracture_l2_logreg_model = LogisticRegressionCV(
  penalty='l2',
  solver='liblinear',
  Cs=10,
  max_iter=10000,
  n_jobs=-1,
  cv=5
).fit(fracture_X_train, fracture_y_train.ravel())

joblib.dump(fracture_l2_logreg_model, fracture_l2_logreg_model_path)

y_pred_logreg = fracture_l2_logreg_model.predict(fracture_X_valid)
print("Fracture L2 logistic Regression:")
print(classification_report(
    fracture_y_valid.ravel(),
    y_pred_logreg,
    labels=[1],
    target_names=["Fracture"],
    zero_division=0
))

Fracture L2 logistic Regression:
              precision    recall  f1-score   support

    Fracture       0.00      0.00      0.00       292

   micro avg       0.00      0.00      0.00       292
   macro avg       0.00      0.00      0.00       292
weighted avg       0.00      0.00      0.00       292



In [50]:
support_devices_l2_logreg_model = LogisticRegressionCV(
    penalty='l2',
    solver='liblinear',
    Cs=10,
    max_iter=10000,
    n_jobs=-1,
    cv=5
).fit(support_devices_X_train, support_devices_y_train.ravel())

joblib.dump(support_devices_l2_logreg_model, support_devices_l2_logreg_model_path)

y_pred_logreg = support_devices_l2_logreg_model.predict(support_devices_X_valid)
print("Support devices L2 logistic Regression:")
print(classification_report(
  support_devices_y_valid.ravel(),
  y_pred_logreg,
  labels=[1],
  target_names=["Support devices"],
  zero_division=0))

Support devices L2 logistic Regression:
                 precision    recall  f1-score   support

Support devices       0.43      0.11      0.18       572

      micro avg       0.43      0.11      0.18       572
      macro avg       0.43      0.11      0.18       572
   weighted avg       0.43      0.11      0.18       572



In [39]:
elasticnetratios = [0.3, 0.5, 0.7]

In [40]:
lung_diseases_elasticnet_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
    penalty='elasticnet',
    solver='saga',  # Elastic Net requires 'saga' solver
    Cs=10,  # Number of C values to try
    l1_ratios=elasticnetratios,
    max_iter=10000,
    n_jobs=-1,
    cv=5
  )
).fit(lung_diseases_X_train, lung_diseases_y_train)

joblib.dump(lung_diseases_elasticnet_logreg_model, lung_diseases_elasticnet_logreg_model_path)

y_pred_logreg = lung_diseases_elasticnet_logreg_model.predict(lung_diseases_X_valid)
print("Lung disease elasticnet logistic Regression:")
print(classification_report(lung_diseases_y_valid, y_pred_logreg, zero_division=0, target_names=lung_diseases_valid.pathologies))

Lung disease elasticnet logistic Regression:
               precision    recall  f1-score   support

  Atelectasis       0.52      0.16      0.25      1211
Consolidation       0.00      0.00      0.00       243
        Edema       0.55      0.14      0.23       356
 Lung Opacity       0.53      0.09      0.15      1612
    Pneumonia       0.64      0.04      0.08       685
  Lung Lesion       0.00      0.00      0.00       392

    micro avg       0.53      0.09      0.16      4499
    macro avg       0.37      0.07      0.12      4499
 weighted avg       0.47      0.09      0.15      4499
  samples avg       0.03      0.02      0.02      4499



In [41]:
pleural_related_elasticnet_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
    penalty='elasticnet',
    solver='saga',  # Elastic Net requires 'saga' solver
    Cs=10,  # Number of C values to try
    l1_ratios=elasticnetratios,
    max_iter=10000,
    n_jobs=-1,
    cv=5
  )
).fit(pleural_related_X_train, pleural_related_y_train)

joblib.dump(pleural_related_elasticnet_logreg_model, pleural_related_elasticnet_logreg_model_path)

y_pred_logreg = pleural_related_elasticnet_logreg_model.predict(pleural_related_X_valid)
print("Pleural related elasticnet logistic Regression:")
print(classification_report(pleural_related_y_valid, y_pred_logreg, zero_division=0, target_names=pleural_related_valid.pathologies))

Pleural related elasticnet logistic Regression:
                  precision    recall  f1-score   support

Pleural Effusion       0.67      0.45      0.54      1121
   Pleural Other       0.00      0.00      0.00       123
    Pneumothorax       0.00      0.00      0.00       175

       micro avg       0.67      0.36      0.47      1419
       macro avg       0.22      0.15      0.18      1419
    weighted avg       0.53      0.36      0.43      1419
     samples avg       0.04      0.03      0.04      1419



In [42]:
cardiac_mediastinal_elasticnet_logreg_model = MultiOutputClassifier(
  LogisticRegressionCV(
    penalty='elasticnet',
    solver='saga',  # Elastic Net requires 'saga' solver
    Cs=10,  # Number of C values to try
    l1_ratios=elasticnetratios,
    max_iter=10000,
    n_jobs=-1,
    cv=5
  )
).fit(cardiac_mediastinal_X_train, cardiac_mediastinal_y_train)

joblib.dump(cardiac_mediastinal_elasticnet_logreg_model, cardiac_mediastinal_elasticnet_logreg_model_path)

y_pred_logreg = cardiac_mediastinal_elasticnet_logreg_model.predict(cardiac_mediastinal_X_valid)
print("Cardiac mediastinal elasticnet logistic Regression:")
print(classification_report(cardiac_mediastinal_y_valid, y_pred_logreg, zero_division=0, target_names=cardiac_mediastinal_valid.pathologies))

Cardiac mediastinal elasticnet logistic Regression:
                            precision    recall  f1-score   support

              Cardiomegaly       0.54      0.19      0.28      1085
Enlarged Cardiomediastinum       0.00      0.00      0.00       189

                 micro avg       0.54      0.16      0.24      1274
                 macro avg       0.27      0.09      0.14      1274
              weighted avg       0.46      0.16      0.23      1274
               samples avg       0.01      0.01      0.01      1274



In [51]:
fracture_elasticnet_logreg_model = LogisticRegressionCV(
  penalty='elasticnet',
  solver='saga',  # Elastic Net requires 'saga' solver
  Cs=10,  # Number of C values to try
  l1_ratios=elasticnetratios,
  max_iter=10000,
  n_jobs=-1,
  cv=5
).fit(fracture_X_train, fracture_y_train.ravel())

joblib.dump(fracture_elasticnet_logreg_model, fracture_elasticnet_logreg_model_path)

y_pred_logreg = fracture_elasticnet_logreg_model.predict(fracture_X_valid)
print("Fracture elasticnet logistic Regression:")
print(classification_report(
    fracture_y_valid.ravel(),
    y_pred_logreg,
    labels=[1],
    target_names=["Fracture"],
    zero_division=0
))

Fracture elasticnet logistic Regression:
              precision    recall  f1-score   support

    Fracture       0.80      0.01      0.03       292

   micro avg       0.80      0.01      0.03       292
   macro avg       0.80      0.01      0.03       292
weighted avg       0.80      0.01      0.03       292



In [52]:
support_devices_elasticnet_logreg_model = LogisticRegressionCV(
    penalty='elasticnet',
    solver='saga',  # Elastic Net requires 'saga' solver
    Cs=10,  # Number of C values to try
    l1_ratios=elasticnetratios,
    max_iter=10000,
    n_jobs=-1,
    cv=5
).fit(support_devices_X_train, support_devices_y_train.ravel())

joblib.dump(support_devices_elasticnet_logreg_model, support_devices_elasticnet_logreg_model_path)

y_pred_logreg = support_devices_elasticnet_logreg_model.predict(support_devices_X_valid)
print("Support devices elasticnet logistic Regression:")
print(classification_report(
  support_devices_y_valid.ravel(),
  y_pred_logreg,
  labels=[1],
  target_names=["Support devices"],
  zero_division=0))

Support devices elasticnet logistic Regression:
                 precision    recall  f1-score   support

Support devices       0.51      0.04      0.08       572

      micro avg       0.51      0.04      0.08       572
      macro avg       0.51      0.04      0.08       572
   weighted avg       0.51      0.04      0.08       572



In [6]:
lung_diseases_vanilla_logreg_model = joblib.load(lung_diseases_vanilla_logreg_model_path)
lung_diseases_l1_logreg_model = joblib.load(lung_diseases_l1_logreg_model_path)
lung_diseases_l2_logreg_model = joblib.load(lung_diseases_l2_logreg_model_path)
lung_diseases_elasticnet_logreg_model = joblib.load(lung_diseases_elasticnet_logreg_model_path)

pleural_related_vanilla_logreg_model = joblib.load(pleural_related_vanilla_logreg_model_path)
pleural_related_l1_logreg_model = joblib.load(pleural_related_l1_logreg_model_path)
pleural_related_l2_logreg_model = joblib.load(pleural_related_l2_logreg_model_path)
pleural_related_elasticnet_logreg_model = joblib.load(pleural_related_elasticnet_logreg_model_path)

cardiac_mediastinal_vanilla_logreg_model = joblib.load(cardiac_mediastinal_vanilla_logreg_model_path)
cardiac_mediastinal_l1_logreg_model = joblib.load(cardiac_mediastinal_l1_logreg_model_path)
cardiac_mediastinal_l2_logreg_model = joblib.load(cardiac_mediastinal_l2_logreg_model_path)
cardiac_mediastinal_elasticnet_logreg_model = joblib.load(cardiac_mediastinal_elasticnet_logreg_model_path)

fracture_vanilla_logreg_model = joblib.load(fracture_vanilla_logreg_model_path)
fracture_l1_logreg_model = joblib.load(fracture_l1_logreg_model_path)
fracture_l2_logreg_model = joblib.load(fracture_l2_logreg_model_path)
fracture_elasticnet_logreg_model = joblib.load(fracture_elasticnet_logreg_model_path)

support_devices_vanilla_logreg_model = joblib.load(support_devices_vanilla_logreg_model_path)
support_devices_l1_logreg_model = joblib.load(support_devices_l1_logreg_model_path)
support_devices_l2_logreg_model = joblib.load(support_devices_l2_logreg_model_path)
support_devices_elasticnet_logreg_model = joblib.load(support_devices_elasticnet_logreg_model_path)

### Test for lung disease dataset

In [7]:
lung_diseases_y_pred_logreg_test = lung_diseases_vanilla_logreg_model.predict(lung_diseases_X_test)
print("Vanilla lung disease Logistic Regression test:")
print(classification_report(lung_diseases_y_test, lung_diseases_y_pred_logreg_test, zero_division=0, target_names=lung_diseases_test.pathologies))

Vanilla lung disease Logistic Regression test:
               precision    recall  f1-score   support

  Atelectasis       0.42      0.25      0.31       855
Consolidation       0.07      0.17      0.10       166
        Edema       0.16      0.26      0.20       229
 Lung Opacity       0.39      0.20      0.27      1120
    Pneumonia       0.28      0.12      0.17       431
  Lung Lesion       0.16      0.10      0.12       271

    micro avg       0.28      0.20      0.23      3072
    macro avg       0.25      0.18      0.20      3072
 weighted avg       0.33      0.20      0.24      3072
  samples avg       0.05      0.04      0.04      3072



In [8]:
lung_diseases_y_pred_l1_logreg_test = lung_diseases_l1_logreg_model.predict(lung_diseases_X_test)
print("L1 lung disease Logistic Regression test:")
print(classification_report(lung_diseases_y_test, lung_diseases_y_pred_l1_logreg_test, zero_division=0, target_names=lung_diseases_test.pathologies))

L1 lung disease Logistic Regression test:
               precision    recall  f1-score   support

  Atelectasis       0.52      0.14      0.22       855
Consolidation       0.00      0.00      0.00       166
        Edema       0.47      0.09      0.15       229
 Lung Opacity       0.53      0.12      0.20      1120
    Pneumonia       0.81      0.05      0.09       431
  Lung Lesion       0.00      0.00      0.00       271

    micro avg       0.53      0.10      0.16      3072
    macro avg       0.39      0.07      0.11      3072
 weighted avg       0.49      0.10      0.16      3072
  samples avg       0.03      0.02      0.02      3072



In [9]:
lung_diseases_y_pred_l2_logreg_test = lung_diseases_l2_logreg_model.predict(lung_diseases_X_test)
print("L1 lung disease Logistic Regression test:")
print(classification_report(lung_diseases_y_test, lung_diseases_y_pred_l2_logreg_test, zero_division=0, target_names=lung_diseases_test.pathologies))

L1 lung disease Logistic Regression test:
               precision    recall  f1-score   support

  Atelectasis       0.51      0.17      0.25       855
Consolidation       0.00      0.00      0.00       166
        Edema       0.54      0.13      0.20       229
 Lung Opacity       0.55      0.11      0.19      1120
    Pneumonia       0.71      0.03      0.05       431
  Lung Lesion       0.00      0.00      0.00       271

    micro avg       0.53      0.10      0.17      3072
    macro avg       0.38      0.07      0.12      3072
 weighted avg       0.48      0.10      0.16      3072
  samples avg       0.03      0.02      0.03      3072



In [10]:
lung_diseases_y_pred_elasticnet_logreg_test = lung_diseases_elasticnet_logreg_model.predict(lung_diseases_X_test)
print("L1 lung disease Logistic Regression test:")
print(classification_report(lung_diseases_y_test, lung_diseases_y_pred_elasticnet_logreg_test, zero_division=0, target_names=lung_diseases_test.pathologies))

L1 lung disease Logistic Regression test:
               precision    recall  f1-score   support

  Atelectasis       0.51      0.16      0.24       855
Consolidation       0.00      0.00      0.00       166
        Edema       0.49      0.14      0.22       229
 Lung Opacity       0.55      0.08      0.13      1120
    Pneumonia       0.70      0.05      0.10       431
  Lung Lesion       0.00      0.00      0.00       271

    micro avg       0.53      0.09      0.15      3072
    macro avg       0.38      0.07      0.12      3072
 weighted avg       0.48      0.09      0.15      3072
  samples avg       0.03      0.02      0.02      3072



### Test for pleural related dataset

In [11]:
pleural_related_vanilla_logreg_test = pleural_related_vanilla_logreg_model.predict(pleural_related_X_test)
print("Vanilla pleural related Logistic Regression test:")
print(classification_report(pleural_related_y_test, pleural_related_vanilla_logreg_test, zero_division=0, target_names=pleural_related_test.pathologies))

Vanilla pleural related Logistic Regression test:
                  precision    recall  f1-score   support

Pleural Effusion       0.55      0.47      0.51       742
   Pleural Other       0.09      0.15      0.11        81
    Pneumothorax       0.11      0.16      0.13       105

       micro avg       0.41      0.41      0.41       928
       macro avg       0.25      0.26      0.25       928
    weighted avg       0.46      0.41      0.43       928
     samples avg       0.04      0.04      0.04       928



In [12]:
pleural_related_l1_logreg_test = pleural_related_l1_logreg_model.predict(pleural_related_X_test)
print("L1 pleural related Logistic Regression test:")
print(classification_report(pleural_related_y_test, pleural_related_l1_logreg_test, zero_division=0, target_names=pleural_related_test.pathologies))

L1 pleural related Logistic Regression test:
                  precision    recall  f1-score   support

Pleural Effusion       0.66      0.50      0.57       742
   Pleural Other       0.00      0.00      0.00        81
    Pneumothorax       0.00      0.00      0.00       105

       micro avg       0.66      0.40      0.50       928
       macro avg       0.22      0.17      0.19       928
    weighted avg       0.53      0.40      0.45       928
     samples avg       0.04      0.04      0.04       928



In [13]:
pleural_related_l2_logreg_test = pleural_related_l2_logreg_model.predict(pleural_related_X_test)
print("L2 pleural related Logistic Regression test:")
print(classification_report(pleural_related_y_test, pleural_related_l2_logreg_test, zero_division=0, target_names=pleural_related_test.pathologies))

L2 pleural related Logistic Regression test:
                  precision    recall  f1-score   support

Pleural Effusion       0.66      0.51      0.57       742
   Pleural Other       0.00      0.00      0.00        81
    Pneumothorax       0.00      0.00      0.00       105

       micro avg       0.66      0.41      0.50       928
       macro avg       0.22      0.17      0.19       928
    weighted avg       0.52      0.41      0.46       928
     samples avg       0.04      0.04      0.04       928



In [14]:
pleural_related_elasticnet_logreg_test = pleural_related_elasticnet_logreg_model.predict(pleural_related_X_test)
print("L2 pleural related Logistic Regression test:")
print(classification_report(pleural_related_y_test, pleural_related_elasticnet_logreg_test, zero_division=0, target_names=pleural_related_test.pathologies))

L2 pleural related Logistic Regression test:
                  precision    recall  f1-score   support

Pleural Effusion       0.66      0.51      0.57       742
   Pleural Other       0.00      0.00      0.00        81
    Pneumothorax       0.00      0.00      0.00       105

       micro avg       0.66      0.41      0.50       928
       macro avg       0.22      0.17      0.19       928
    weighted avg       0.53      0.41      0.46       928
     samples avg       0.04      0.04      0.04       928



### Test for cardiac mediastinal dataset

In [15]:
cardiac_mediastinal_vanilla_logreg_test = cardiac_mediastinal_vanilla_logreg_model.predict(cardiac_mediastinal_X_test)
print("Vanilla cardiac mediastinal Logistic Regression test:")
print(classification_report(cardiac_mediastinal_y_test, cardiac_mediastinal_vanilla_logreg_test, zero_division=0, target_names=cardiac_mediastinal_test.pathologies))

Vanilla cardiac mediastinal Logistic Regression test:
                            precision    recall  f1-score   support

              Cardiomegaly       0.42      0.26      0.32       761
Enlarged Cardiomediastinum       0.05      0.09      0.07       139

                 micro avg       0.29      0.24      0.26       900
                 macro avg       0.23      0.18      0.19       900
              weighted avg       0.36      0.24      0.28       900
               samples avg       0.02      0.02      0.02       900



In [16]:
cardiac_mediastinal_l1_logreg_test = cardiac_mediastinal_l1_logreg_model.predict(cardiac_mediastinal_X_test)
print("L1 cardiac mediastinal Logistic Regression test:")
print(classification_report(cardiac_mediastinal_y_test, cardiac_mediastinal_l1_logreg_test, zero_division=0, target_names=cardiac_mediastinal_test.pathologies))

L1 cardiac mediastinal Logistic Regression test:
                            precision    recall  f1-score   support

              Cardiomegaly       0.61      0.20      0.30       761
Enlarged Cardiomediastinum       0.00      0.00      0.00       139

                 micro avg       0.61      0.17      0.26       900
                 macro avg       0.31      0.10      0.15       900
              weighted avg       0.52      0.17      0.25       900
               samples avg       0.02      0.02      0.02       900



In [17]:
cardiac_mediastinal_l2_logreg_test = cardiac_mediastinal_l2_logreg_model.predict(cardiac_mediastinal_X_test)
print("L2 cardiac mediastinal Logistic Regression test:")
print(classification_report(cardiac_mediastinal_y_test, cardiac_mediastinal_l2_logreg_test, zero_division=0, target_names=cardiac_mediastinal_test.pathologies))

L2 cardiac mediastinal Logistic Regression test:
                            precision    recall  f1-score   support

              Cardiomegaly       0.67      0.08      0.15       761
Enlarged Cardiomediastinum       0.00      0.00      0.00       139

                 micro avg       0.67      0.07      0.12       900
                 macro avg       0.33      0.04      0.07       900
              weighted avg       0.56      0.07      0.12       900
               samples avg       0.01      0.01      0.01       900



In [18]:
cardiac_mediastinal_elasticnet_logreg_test = cardiac_mediastinal_elasticnet_logreg_model.predict(cardiac_mediastinal_X_test)
print("Elasticnet cardiac mediastinal Logistic Regression test:")
print(classification_report(cardiac_mediastinal_y_test, cardiac_mediastinal_elasticnet_logreg_test, zero_division=0, target_names=cardiac_mediastinal_test.pathologies))

Elasticnet cardiac mediastinal Logistic Regression test:
                            precision    recall  f1-score   support

              Cardiomegaly       0.60      0.19      0.29       761
Enlarged Cardiomediastinum       0.00      0.00      0.00       139

                 micro avg       0.60      0.16      0.26       900
                 macro avg       0.30      0.10      0.15       900
              weighted avg       0.51      0.16      0.25       900
               samples avg       0.02      0.02      0.02       900



### Test for fracture dataset

In [19]:
fracture_vanilla_logreg_test = fracture_vanilla_logreg_model.predict(fracture_X_test)
print("Vanilla cardiac mediastinal Logistic Regression test:")
print(classification_report(
    fracture_y_test.ravel(),
    fracture_vanilla_logreg_test,
    labels=[1],
    target_names=["Fracture"],
    zero_division=0
))

Vanilla cardiac mediastinal Logistic Regression test:
              precision    recall  f1-score   support

    Fracture       0.07      0.06      0.06       184

   micro avg       0.07      0.06      0.06       184
   macro avg       0.07      0.06      0.06       184
weighted avg       0.07      0.06      0.06       184



In [20]:
fracture_l1_logreg_test = fracture_l1_logreg_model.predict(fracture_X_test)
print("L1 cardiac mediastinal Logistic Regression test:")
print(classification_report(
    fracture_y_test.ravel(),
    fracture_l1_logreg_test,
    labels=[1],
    target_names=["Fracture"],
    zero_division=0
))

L1 cardiac mediastinal Logistic Regression test:
              precision    recall  f1-score   support

    Fracture       0.00      0.00      0.00       184

   micro avg       0.00      0.00      0.00       184
   macro avg       0.00      0.00      0.00       184
weighted avg       0.00      0.00      0.00       184



In [21]:
fracture_l2_logreg_test = fracture_l2_logreg_model.predict(fracture_X_test)
print("L2 cardiac mediastinal Logistic Regression test:")
print(classification_report(
    fracture_y_test.ravel(),
    fracture_l2_logreg_test,
    labels=[1],
    target_names=["Fracture"],
    zero_division=0
))

L2 cardiac mediastinal Logistic Regression test:
              precision    recall  f1-score   support

    Fracture       0.00      0.00      0.00       184

   micro avg       0.00      0.00      0.00       184
   macro avg       0.00      0.00      0.00       184
weighted avg       0.00      0.00      0.00       184



In [22]:
fracture_elasticnet_logreg_test = fracture_elasticnet_logreg_model.predict(fracture_X_test)
print("Elasticnet cardiac mediastinal Logistic Regression test:")
print(classification_report(
    fracture_y_test.ravel(),
    fracture_elasticnet_logreg_test,
    labels=[1],
    target_names=["Fracture"],
    zero_division=0
))

Elasticnet cardiac mediastinal Logistic Regression test:
              precision    recall  f1-score   support

    Fracture       0.00      0.00      0.00       184

   micro avg       0.00      0.00      0.00       184
   macro avg       0.00      0.00      0.00       184
weighted avg       0.00      0.00      0.00       184



### Test for support devices dataset

In [23]:
support_devices_vanilla_logreg_test = support_devices_vanilla_logreg_model.predict(support_devices_X_test)
print("Vanilla cardiac mediastinal Logistic Regression test:")
print(classification_report(
    support_devices_y_test.ravel(),
    support_devices_vanilla_logreg_test,
    labels=[1],
    target_names=["support_devices"],
    zero_division=0
))

Vanilla cardiac mediastinal Logistic Regression test:
                 precision    recall  f1-score   support

support_devices       0.30      0.24      0.26       398

      micro avg       0.30      0.24      0.26       398
      macro avg       0.30      0.24      0.26       398
   weighted avg       0.30      0.24      0.26       398



In [24]:
support_devices_l1_logreg_test = support_devices_l1_logreg_model.predict(support_devices_X_test)
print("L1 cardiac mediastinal Logistic Regression test:")
print(classification_report(
    support_devices_y_test.ravel(),
    support_devices_l1_logreg_test,
    labels=[1],
    target_names=["support_devices"],
    zero_division=0
))

L1 cardiac mediastinal Logistic Regression test:
                 precision    recall  f1-score   support

support_devices       0.43      0.07      0.12       398

      micro avg       0.43      0.07      0.12       398
      macro avg       0.43      0.07      0.12       398
   weighted avg       0.43      0.07      0.12       398



In [25]:
support_devices_l2_logreg_test = support_devices_l2_logreg_model.predict(support_devices_X_test)
print("L2 cardiac mediastinal Logistic Regression test:")
print(classification_report(
    support_devices_y_test.ravel(),
    support_devices_l2_logreg_test,
    labels=[1],
    target_names=["support_devices"],
    zero_division=0
))

L2 cardiac mediastinal Logistic Regression test:
                 precision    recall  f1-score   support

support_devices       0.46      0.11      0.17       398

      micro avg       0.46      0.11      0.17       398
      macro avg       0.46      0.11      0.17       398
   weighted avg       0.46      0.11      0.17       398



In [26]:
support_devices_elasticnet_logreg_test = support_devices_elasticnet_logreg_model.predict(support_devices_X_test)
print("Elasticnet cardiac mediastinal Logistic Regression test:")
print(classification_report(
    support_devices_y_test.ravel(),
    support_devices_elasticnet_logreg_test,
    labels=[1],
    target_names=["support_devices"],
    zero_division=0
))

Elasticnet cardiac mediastinal Logistic Regression test:
                 precision    recall  f1-score   support

support_devices       0.47      0.04      0.08       398

      micro avg       0.47      0.04      0.08       398
      macro avg       0.47      0.04      0.08       398
   weighted avg       0.47      0.04      0.08       398

